In [15]:
from Models.models import LLMModel
from langchain_core.messages import (HumanMessage)
import json
from Models.schemas import AspectorEvaluatorInputSchema, FeedbackISC, FeedbackBasic,  QualityAspect, AspectorRole

In [16]:
model1 = LLMModel(
    provider="openai_api",
    model="gpt-4-1106-preview",
    output_schema=FeedbackBasic,
    chat_history=[],
    use_history= True,
    )


model2 = LLMModel(
    provider="fireworks_api",
    model="llama-v3-70b-instruct",
    output_schema=FeedbackBasic,
    chat_history=[],
    use_history= True,
    )


model3 = LLMModel(
    provider="anthropic_api",
    model="claude-3-opus-20240229",
    output_schema=FeedbackBasic,
    chat_history=[],
    use_history= True,
    )


In [17]:
this_model = model1

messages=[HumanMessage(content="How do you say up in french "),]
response = this_model(messages)
print(json.loads(response.content))

print("--------------------------------")


messages=[HumanMessage(content="What was my first message."),]
response = this_model(messages)
print(json.loads(response.content))


print("--------------------------------")



for message in this_model.chat_history:
    print(message)
   
    
print("--------------------------------")


tokens, all_message = this_model.get_total_tokens()
print(tokens)

{'response': 'en haut'}
--------------------------------
{'response': 'How do you say up in french'}
--------------------------------
content='How do you say up in french '
content='{\n    "response": "en haut"\n}' response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 175, 'total_tokens': 185}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-71ff7ff2-a0b7-4ec1-9f8f-d33dc9ff869e-0'
content='What was my first message.'
content='{\n    "response": "How do you say up in french"\n}' response_metadata={'token_usage': {'completion_tokens': 15, 'prompt_tokens': 199, 'total_tokens': 214}, 'model_name': 'gpt-4-1106-preview', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-a49cbe56-a630-43b5-8ccd-6589e2d9431e-0'
--------------------------------
214


#### LLM AS EVALUATOR


In [18]:

    
# Example usage of LLMasEvaluator
evaluator = LLMModel(
    provider="anthropic_api",
    model="claude-3-opus-20240229",
    output_schema=FeedbackISC,
    input_schema=AspectorEvaluatorInputSchema,
    name="aspect_evaluator",
    chat_history=[],
    use_history=False,
    as_evaluator = True
)

convo = [
    "user: Hello, are you doing today?",
    "assistant: I'm doing well, thank you for asking! How can I assist you today?",
    "user: I was hoping you could provide some feedback, on a short essay. I wrote. Would you be able to take a look and let me know your thoughts on the grammar and structure?",
    "assistant: Absolutely, I'd be happy to review, your essay and provide feedback on the grammar and overall structure. Please go ahead and send over the essay whenever you're ready, and I'll take a look. Let me know if there are any specific areas you'd like me to focus on in addition to the grammar and structure."
    ]


quality_aspect = QualityAspect(
    name="grammar",
    instruction="The quality of the grammar including how well it is structured and punctuated"
)


# Create an instance of AspectorEvaluatorInputSchema
evaluation_task = AspectorEvaluatorInputSchema(
    quality_aspect=quality_aspect,
    role=AspectorRole.USER_AND_ASSISTANT,
    conversation=convo,
    metadata = {}

)


# Perform evaluation
evaluation_result = json.loads(evaluator(evaluation_task).content)

print(f"score: {evaluation_result["score"]}")
print(f"issues: {evaluation_result["issues"]}")
print(f"comment: {evaluation_result["comment"]}")



score: 3
issues: ["Missing word in user's first message", "Incorrect comma placement in assistant's second message", "Run-on sentence in user's second message"]
comment: The grammar in this conversation has a few issues. The user's messages are missing some words and have run-on sentences. The assistant's message has an incorrectly placed comma. However, the messages are still understandable overall. Some proofreading and editing would help improve the grammatical quality.


In [ ]:
token,system_message = evaluator.get_total_tokens()
print(b[0].content)